**IMPORT STATMENTS**

In [1]:
# Add import statements here

import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup  

**CREATING FUNCTION TO SCRAPE REVIEWS**

In [2]:
def getFullReviews(page_url):
    
    reviews=[]  # variable to hold all reviews
    rows=[]
    df = pd.DataFrame()
    domain_name = 'https://www.consumeraffairs.com/'
    
    while page_url:
        
        page = requests.get(page_url)
        
        if page.status_code==200:
            soup = BeautifulSoup(page.content, 'html.parser')
            divs = soup.select("div.js-paginator-data div.rvw.js-rvw") 
            #print(divs)
            
            for idx, div in enumerate(divs):

                reviewer = None
                content = None
                date = None
                ratings = None

    #### getting Reviewer name ####
    
                a_reviewer = div.select("div.rvw-aut strong.rvw-aut__inf-nm")
        
                if a_reviewer!=[]:
                    reviewer=a_reviewer[0].get_text()
                    reviewer = reviewer[:-4]
                #print(reviewer)
            
    #### Getting Content ####
            
                a_content = div.select("div.rvw-bd")   
                #print(a_content)
    
                if a_content!=[]:
                    content=a_content[0].get_text()
                #print(content)
    
                if "review" in content:
                    content = content.replace("Read full review", " ")
                    content = content.replace("Original review:", " ")
                #print(content)
        
                d = re.findall("\w+[ .]*\d+[, ] \d+", content)
                #print(d)

                for dates in d:
                     content = content.replace(dates," ")
                #print(content)
    
        #### Getting date ####
    
                a_date = div.select("div.rvw-bd")
        
                if a_date!=[]:
                    date=a_date[0].get_text()
                #print(date)
        
                date = re.findall("\w+[ .]*\d+[, ] \d+", date)
                #print(date)
    
                for dates in date:
                    date = dates
                #print(date)
    
        #### Getting rating ####
    
                a_rating = div.select("div.rvw.js-rvw img.stars-rtg")  #div.rvw.js-rvw
    
                rating = ""
                
                for element in a_rating:
                    rating += str(element)
                #print(ratings)
    
                rating = re.findall(r'(data-rating="[0-9]\.[0-9]")', rating)
                rating =re.findall(r'([0-9]\.[0-9])', str(rating))
                #print(rating)
    
                for ratings in rating:
                     rating = ratings
                
                if ratings in rating:
                    rating = ratings
                else:
                    rating = None
                    
                #print(rating)
                
                rows.append((date,reviewer, content, rating)) # 
                #print(rows)     
     
            next_btn = soup.select("div.h-col-stk a.ca-a-md.js-pager-next")
            #print(next_btn)
            
            if len(next_btn)>0:
                next_page = next_btn[0]["href"]
                if domain_name in next_page:
                    next_page = next_page.replace(domain_name, "")
                    #print(next_page)
                    page_url = domain_name + next_page
                    #print(page_url)   
            else:
                page_url = None
            
    df = pd.DataFrame(rows, columns = ["Date", "Reviewer","Content", "Rating" ])
    #print(df)
    reviews = df.apply(lambda r: r.str.strip() if r.dtype == "object" else r)   
    
    
    return(reviews)

**SCRAPING DATA FOR AT&T AND DOWNLOADING DATASET**

In [5]:
getFullReviews('https://www.consumeraffairs.com/cell_phones/att_wireless.htm').to_csv("AT&T_Reviews.csv", index = False)

**SCRAPING DATA FOR VERIZON AND DOWNLOADING DATASET**

In [6]:
getFullReviews('https://www.consumeraffairs.com/cell_phones/verizon_wireless.html').to_csv("Verizon_Reviews.csv", index = False)